In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras import backend as K

import sys
sys.path.append("..")
from src.model_architectures.unet_models import MODELS
import glob

In [2]:
folder_name = 'sh06x2'
train_exe = 50
train_x = np.load(f'../DATASETS/{folder_name}/lab_ch0_shapes_train_x_m_{train_exe}.npy')
train_y = np.load(f'../DATASETS/{folder_name}/lab_ch0_shapes_train_y_m_{train_exe}.npy')

val_x = np.load(f'../DATASETS/{folder_name}/lab_ch0_shapes_val_x_m_{train_exe}.npy')
val_y = np.load(f'../DATASETS/{folder_name}/lab_ch0_shapes_val_y_m_{train_exe}.npy')

test_x = np.load(f'../DATASETS/{folder_name}/lab_ch0_shapes_test_x_m_{train_exe}.npy')
test_y = np.load(f'../DATASETS/{folder_name}/lab_ch0_shapes_test_y_m_{train_exe}.npy')

print(train_x.shape,val_x.shape,test_x.shape)
print(train_y.shape,val_y.shape,test_y.shape)

(3000, 256) (960, 256) (960, 256)
(3000, 256) (960, 256) (960, 256)


In [9]:
bins = 256
channel = 1
loss_func = 'mse'
lr = '1e-4'
opt = tf.keras.optimizers.Nadam(learning_rate=float(lr))
metric = 'mae'
reg = '0'
filt_lst = [64,128,256,512]
dns = 256
drop = '0'
batch_size = 256#

ep=100

# vgg16, unet, resUnet, att_resUnet, unet++
get_model = MODELS()
model_type="Unet16"

if model_type=="attention_resunet":
    FILTER_NUM = 16
    FILTER_SIZE = 3
    NUM_CLASSES = 1
    dropout_rate = 0.2
    batch_norm = False
    model = get_model.Attention_ResUNet(bins,channel,loss_func,opt,metric,FILTER_NUM=FILTER_NUM,batch_norm = False, dropout_rate = float(drop))
elif model_type=="unetpp":
    nb_filter = [16, 32, 64, 128, 256]
    deep_supervision = False
    model = get_model.Nest_Net(bins,channel,loss_func,opt,metric,nb_filter=nb_filter,deep_supervision=deep_supervision)
elif model_type=="Unet16":
    filt_num = 16
    model = get_model.UNET(bins, channel, loss_func, opt, metric, float(reg), filt_num)
elif model_type=="vgg16":
    model = get_model.VGG16(filt_lst,dns,bins,channel,loss_func,opt,metric)
elif model_type=="resUnet":
    model = get_model.resUnet(bins, channel, loss_func, opt, metric, float(reg))
elif model_type=="unetPP48163264":
    nb_filter = [4,8,16,32,64]
    filters = [4,8,16,32,64]
    model = get_model.unetPP(bins,channel,loss_func,opt,metric,float(reg), nb_filter, filters)



if len(train_x.shape)<3:
    train_X = train_x.reshape(train_x.shape[0],train_x.shape[1],1)
    val_X = val_x.reshape(val_x.shape[0],val_x.shape[1],1)
    test_X = test_x.reshape(test_x.shape[0],test_x.shape[1],1)
    
    train_y = train_y.reshape(train_y.shape[0],train_y.shape[1],1)
    val_y = val_y.reshape(val_y.shape[0],val_y.shape[1],1)
    test_y = test_y.reshape(test_y.shape[0],test_y.shape[1],1)

folder_name = 'sh06x2'
chnl = 'ch0'
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, verbose=1)
mc = tf.keras.callbacks.ModelCheckpoint(f'../models/{folder_name}/{model_type}_{folder_name}_{chnl}__256_m_1280_{loss_func}_lr_{lr}_{ep}_DROPOUT{reg}.h5',
                                       monitor='val_loss', model='min', verbose=1, save_best_only=True)


history=model.fit(train_X[:],train_y[:],
                batch_size=batch_size,epochs=ep,
                validation_data=(val_X[:],val_y[:]),shuffle=True, callbacks=[callback, mc])

layer output: (None, 256, 1)
Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 256, 1)]     0           []                               
                                                                                                  
 conv1d_76 (Conv1D)             (None, 256, 16)      64          ['input_5[0][0]']                
                                                                                                  
 conv1d_77 (Conv1D)             (None, 256, 16)      784         ['conv1d_76[0][0]']              
                                                                                                  
 max_pooling1d_20 (MaxPooling1D  (None, 128, 16)     0           ['conv1d_77[0][0]']              
 )                                                             

12/12 [==============================] - ETA: 0s - loss: 48.2824 - mae: 2.8518
Epoch 1: val_loss improved from inf to 6.61966, saving model to ../models/sh06x2\Unet16_sh06x2_ch0__256_m_1280_mse_lr_1e-4_100_DROPOUT0.h5
12/12 [==============================] - 7s 209ms/step - loss: 48.2824 - mae: 2.8518 - val_loss: 6.6197 - val_mae: 0.8415
Epoch 2/100
12/12 [==============================] - ETA: 0s - loss: 2.7841 - mae: 0.4586
Epoch 2: val_loss improved from 6.61966 to 0.82365, saving model to ../models/sh06x2\Unet16_sh06x2_ch0__256_m_1280_mse_lr_1e-4_100_DROPOUT0.h5
12/12 [==============================] - 1s 47ms/step - loss: 2.7841 - mae: 0.4586 - val_loss: 0.8237 - val_mae: 0.2283
Epoch 3/100
11/12 [==========================>...] - ETA: 0s - loss: 0.5711 - mae: 0.1868
Epoch 3: val_loss improved from 0.82365 to 0.37821, saving model to ../models/sh06x2\Unet16_sh06x2_ch0__256_m_1280_mse_lr_1e-4_100_DROPOUT0.h5
12/12 [==============================] - 0s 40ms/step - loss: 0.5587 - mae

11/12 [==========================>...] - ETA: 0s - loss: 0.1285 - mae: 0.1125
Epoch 24: val_loss improved from 0.12804 to 0.12799, saving model to ../models/sh06x2\Unet16_sh06x2_ch0__256_m_1280_mse_lr_1e-4_100_DROPOUT0.h5
12/12 [==============================] - 0s 40ms/step - loss: 0.1281 - mae: 0.1121 - val_loss: 0.1280 - val_mae: 0.1123
Epoch 25/100
11/12 [==========================>...] - ETA: 0s - loss: 0.1277 - mae: 0.1118
Epoch 25: val_loss improved from 0.12799 to 0.12796, saving model to ../models/sh06x2\Unet16_sh06x2_ch0__256_m_1280_mse_lr_1e-4_100_DROPOUT0.h5
12/12 [==============================] - 1s 44ms/step - loss: 0.1280 - mae: 0.1120 - val_loss: 0.1280 - val_mae: 0.1123
Epoch 26/100
11/12 [==========================>...] - ETA: 0s - loss: 0.1282 - mae: 0.1121
Epoch 26: val_loss improved from 0.12796 to 0.12793, saving model to ../models/sh06x2\Unet16_sh06x2_ch0__256_m_1280_mse_lr_1e-4_100_DROPOUT0.h5
12/12 [==============================] - 0s 42ms/step - loss: 0.1280

11/12 [==========================>...] - ETA: 0s - loss: 0.1276 - mae: 0.1118
Epoch 52: val_loss did not improve from 0.12785
12/12 [==============================] - 0s 33ms/step - loss: 0.1275 - mae: 0.1117 - val_loss: 0.1279 - val_mae: 0.1121
Epoch 53/100
11/12 [==========================>...] - ETA: 0s - loss: 0.1284 - mae: 0.1124
Epoch 53: val_loss improved from 0.12785 to 0.12785, saving model to ../models/sh06x2\Unet16_sh06x2_ch0__256_m_1280_mse_lr_1e-4_100_DROPOUT0.h5
12/12 [==============================] - 0s 42ms/step - loss: 0.1275 - mae: 0.1117 - val_loss: 0.1278 - val_mae: 0.1121
Epoch 54/100
11/12 [==========================>...] - ETA: 0s - loss: 0.1274 - mae: 0.1117
Epoch 54: val_loss did not improve from 0.12785
12/12 [==============================] - 0s 36ms/step - loss: 0.1275 - mae: 0.1117 - val_loss: 0.1279 - val_mae: 0.1121
Epoch 55/100
11/12 [==========================>...] - ETA: 0s - loss: 0.1275 - mae: 0.1117
Epoch 55: val_loss did not improve from 0.12785
1

Epoch 82/100
11/12 [==========================>...] - ETA: 0s - loss: 0.1274 - mae: 0.1117
Epoch 82: val_loss did not improve from 0.12780
12/12 [==============================] - 0s 34ms/step - loss: 0.1273 - mae: 0.1115 - val_loss: 0.1279 - val_mae: 0.1121
Epoch 83/100
11/12 [==========================>...] - ETA: 0s - loss: 0.1275 - mae: 0.1117
Epoch 83: val_loss did not improve from 0.12780
12/12 [==============================] - 0s 34ms/step - loss: 0.1272 - mae: 0.1115 - val_loss: 0.1278 - val_mae: 0.1121
Epoch 84/100
11/12 [==========================>...] - ETA: 0s - loss: 0.1270 - mae: 0.1113
Epoch 84: val_loss did not improve from 0.12780
12/12 [==============================] - 0s 33ms/step - loss: 0.1272 - mae: 0.1115 - val_loss: 0.1278 - val_mae: 0.1121
Epoch 85/100
11/12 [==========================>...] - ETA: 0s - loss: 0.1278 - mae: 0.1120
Epoch 85: val_loss did not improve from 0.12780
12/12 [==============================] - 0s 33ms/step - loss: 0.1272 - mae: 0.1115 -

In [10]:
np.save(f'../model_performance/{folder_name}/loss_{model_type}_{folder_name}_{chnl}__256_m_1280_{loss_func}_lr_{lr}_{ep}_DROPOUT{reg}.npy',
       history.history['loss'])
np.save(f'../model_performance/{folder_name}/val_loss_{model_type}_{folder_name}_{chnl}__256_m_1280_{loss_func}_lr_{lr}_{ep}_DROPOUT{reg}.npy',
       history.history['val_loss'])

np.save(f'../model_performance/{folder_name}/mae_{model_type}_{folder_name}_{chnl}__256_m_1280_{loss_func}_lr_{lr}_{ep}_DROPOUT{reg}.npy',
       history.history['mae'])
np.save(f'../model_performance/{folder_name}/val_mae_{model_type}_{folder_name}_{chnl}__256_m_1280_{loss_func}_lr_{lr}_{ep}_DROPOUT{reg}.npy',
       history.history['val_mae'])

In [12]:
history.history['val_mae']

[1.842753291130066,
 1.1531347036361694,
 0.788569450378418,
 0.5821607708930969,
 0.45612818002700806,
 0.37098509073257446,
 0.3097356855869293,
 0.26459619402885437,
 0.23056954145431519,
 0.20529547333717346]